In [145]:
# Import 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mlp

In [146]:
# read data
df = pd.read_pickle("../../data/raw/df_resample_100ms.pkl")

In [147]:
df

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label,subject_id,trial,age,height,weight,gender
timestamp,,,,,,,,,,,,,
1970-01-01 00:38:38.700,0.948777,-9.636166,0.002699,0.003818,0.016875,0.006643,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:38.800,0.948993,-9.759188,0.087482,0.020693,0.027565,0.015669,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:38.900,1.071514,-9.787465,-0.093610,0.168721,0.075747,0.037797,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:39.000,1.135679,-9.754036,0.065878,0.339488,0.078970,0.040134,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:39.100,1.126174,-9.443248,0.075006,0.594494,-0.012065,0.103557,BSC,1.0,1.0,32.0,180.0,85.0,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 04:49:55.200,2.128992,14.084110,0.907692,-1.022787,0.162796,0.609964,WAL,67.0,1.0,23.0,180.0,67.0,M
1970-01-01 04:49:55.300,-1.563028,11.139874,1.061856,-2.417346,-0.699792,1.861551,WAL,67.0,1.0,23.0,180.0,67.0,M
1970-01-01 04:49:55.400,2.767679,8.869539,0.154884,-0.998184,0.612194,1.557859,WAL,67.0,1.0,23.0,180.0,67.0,M


## 2. Feature Engineering :

In [148]:
df_temporal = df.copy()

In [149]:
# Compute magnitude features
df_temporal['acc_mag'] = np.sqrt(df['acc_x']**2 + df['acc_y']**2 + df['acc_z']**2)
df_temporal['gyro_mag'] = np.sqrt(df['gyro_x']**2 + df['gyro_y']**2 + df['gyro_z']**2)


### 2.1 Temporal based features

In [150]:
# Specify rolling window (100ms dataset, window_size = 10 means roll over every 1 sec)
window_size = 10
stride = 5  # 50% overlap

In [151]:
from scipy.stats import skew, kurtosis

In [152]:
df_temporal

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label,subject_id,trial,age,height,weight,gender,acc_mag,gyro_mag
timestamp,,,,,,,,,,,,,,,
1970-01-01 00:38:38.700,0.948777,-9.636166,0.002699,0.003818,0.016875,0.006643,BSC,1.0,1.0,32.0,180.0,85.0,M,9.682762,0.018533
1970-01-01 00:38:38.800,0.948993,-9.759188,0.087482,0.020693,0.027565,0.015669,BSC,1.0,1.0,32.0,180.0,85.0,M,9.805610,0.037862
1970-01-01 00:38:38.900,1.071514,-9.787465,-0.093610,0.168721,0.075747,0.037797,BSC,1.0,1.0,32.0,180.0,85.0,M,9.846389,0.188767
1970-01-01 00:38:39.000,1.135679,-9.754036,0.065878,0.339488,0.078970,0.040134,BSC,1.0,1.0,32.0,180.0,85.0,M,9.820149,0.350855
1970-01-01 00:38:39.100,1.126174,-9.443248,0.075006,0.594494,-0.012065,0.103557,BSC,1.0,1.0,32.0,180.0,85.0,M,9.510459,0.603567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 04:49:55.200,2.128992,14.084110,0.907692,-1.022787,0.162796,0.609964,WAL,67.0,1.0,23.0,180.0,67.0,M,14.273005,1.201937
1970-01-01 04:49:55.300,-1.563028,11.139874,1.061856,-2.417346,-0.699792,1.861551,WAL,67.0,1.0,23.0,180.0,67.0,M,11.299000,3.130279
1970-01-01 04:49:55.400,2.767679,8.869539,0.154884,-0.998184,0.612194,1.557859,WAL,67.0,1.0,23.0,180.0,67.0,M,9.292619,1.948866


In [153]:
df_temporal_list = []

# Loop through each unique subject_id
for s in df_temporal['subject_id'].unique():
    subject_data = df_temporal[df_temporal['subject_id'] == s].copy()
    
    # Loop through each unique label within the subject
    for l in subject_data['label'].unique():
        # Subset the data for the current subject_id and label, and reset the index
        subset = subject_data[subject_data['label'] == l].copy()
        
        # Loop through each sensor column and compute rolling features
        for col in ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']:
            subset[f'{col}_mean'] = subset[col].rolling(window=10).mean()
            subset[f'{col}_std'] = subset[col].rolling(window=10).std()
            subset[f'{col}_skew'] = subset[col].rolling(window=10).apply(skew, raw=True)
            subset[f'{col}_kurtosis'] = subset[col].rolling(window=10).apply(kurtosis, raw=True)
            subset[f'{col}_jerk'] = subset[col].diff()
            # Compute rolling minimum and maximum values
            subset[f'{col}_min'] = subset[col].rolling(window=10).min()
            subset[f'{col}_max'] = subset[col].rolling(window=10).max()
        
            # Compute the range over the window using min and max difference
            subset[f'{col}_minmax_diff'] = subset[f'{col}_max'] - subset[f'{col}_min']
        
        # Add subject_id and label to the subset explicitly
        subset['subject_id'] = s
        subset['label'] = l
        
        # Append the subset to the list
        df_temporal_list.append(subset)

# Concatenate all subsets into one DataFrame
df_temporal_features = pd.concat(df_temporal_list)


In [154]:
df_temporal_features.columns

Index(['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'label',
       'subject_id', 'trial', 'age', 'height', 'weight', 'gender', 'acc_mag',
       'gyro_mag', 'acc_x_mean', 'acc_x_std', 'acc_x_skew', 'acc_x_kurtosis',
       'acc_x_jerk', 'acc_x_min', 'acc_x_max', 'acc_x_minmax_diff',
       'acc_y_mean', 'acc_y_std', 'acc_y_skew', 'acc_y_kurtosis', 'acc_y_jerk',
       'acc_y_min', 'acc_y_max', 'acc_y_minmax_diff', 'acc_z_mean',
       'acc_z_std', 'acc_z_skew', 'acc_z_kurtosis', 'acc_z_jerk', 'acc_z_min',
       'acc_z_max', 'acc_z_minmax_diff', 'gyro_x_mean', 'gyro_x_std',
       'gyro_x_skew', 'gyro_x_kurtosis', 'gyro_x_jerk', 'gyro_x_min',
       'gyro_x_max', 'gyro_x_minmax_diff', 'gyro_y_mean', 'gyro_y_std',
       'gyro_y_skew', 'gyro_y_kurtosis', 'gyro_y_jerk', 'gyro_y_min',
       'gyro_y_max', 'gyro_y_minmax_diff', 'gyro_z_mean', 'gyro_z_std',
       'gyro_z_skew', 'gyro_z_kurtosis', 'gyro_z_jerk', 'gyro_z_min',
       'gyro_z_max', 'gyro_z_minmax_diff'],
      dt

In [167]:
df_temporal_features.dropna()

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label,subject_id,trial,age,...,gyro_y_max,gyro_y_minmax_diff,gyro_z_mean,gyro_z_std,gyro_z_skew,gyro_z_kurtosis,gyro_z_jerk,gyro_z_min,gyro_z_max,gyro_z_minmax_diff
timestamp,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:38:39.600,0.435292,-7.169052,4.879115,1.259650,0.302332,0.247309,BSC,1.0,1.0,32.0,...,0.441106,0.635926,0.036652,0.101000,0.416793,0.480034,0.317360,-0.132069,0.247309,0.379378
1970-01-01 00:38:39.700,-0.068994,-6.987095,5.288118,1.340131,0.254273,0.553948,BSC,1.0,1.0,32.0,...,0.441106,0.635926,0.091382,0.191064,1.453078,1.594988,0.306639,-0.132069,0.553948,0.686017
1970-01-01 00:38:39.800,-0.260160,-6.908538,6.621654,1.416016,-0.100564,0.402301,BSC,1.0,1.0,32.0,...,0.441106,0.635926,0.130046,0.212011,0.854478,-0.280205,-0.151647,-0.132069,0.553948,0.686017
1970-01-01 00:38:39.900,-0.296217,-4.962242,7.088419,1.469558,-0.128694,0.336724,BSC,1.0,1.0,32.0,...,0.441106,0.635926,0.159938,0.218533,0.452566,-0.881960,-0.065576,-0.132069,0.553948,0.686017
1970-01-01 00:38:40.000,-0.216116,-4.478975,7.648578,1.727389,-0.158978,0.247523,BSC,1.0,1.0,32.0,...,0.441106,0.635926,0.180677,0.215723,0.193332,-0.917896,-0.089202,-0.132069,0.553948,0.686017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 04:49:55.200,2.128992,14.084110,0.907692,-1.022787,0.162796,0.609964,WAL,67.0,1.0,23.0,...,1.065380,2.239997,-0.128272,1.100570,1.237367,0.620927,0.912404,-1.249204,2.374952,3.624157
1970-01-01 04:49:55.300,-1.563028,11.139874,1.061856,-2.417346,-0.699792,1.861551,WAL,67.0,1.0,23.0,...,1.065380,2.239997,-0.179612,0.975716,0.957954,-0.157557,1.251587,-1.249204,1.861551,3.110755
1970-01-01 04:49:55.400,2.767679,8.869539,0.154884,-0.998184,0.612194,1.557859,WAL,67.0,1.0,23.0,...,0.612194,1.786811,-0.104783,1.082822,0.876532,-0.680915,-0.303692,-1.249204,1.861551,3.110755


### Frequency features

Explanation of Frequency Features:

+ FFT (Fast Fourier Transform): Extracts the most dominant frequency component from the signal

+ PSD (Power Spectral Density): Measures the signal's power distribution across different frequencies

+ Spectral Entropy: Represents signal complexity based on frequency distribution

Why This Helps?

+ FFT captures periodicity → Important for detecting abnormal patterns

+ SD gives power information → Differentiates between stable vs. unstable movements

+ Entropy captures randomness → Helps classify smooth vs. erratic motion

In [155]:
from scipy.stats import entropy
from scipy.fft import fft, fftfreq
from scipy.signal import welch


### 2. Compute Frequency-Domain Features ###
def compute_fft(series):
    """Compute the dominant frequency component."""
    fft_vals = np.abs(fft(series))
    return fft_vals[1]  # First dominant frequency component

def compute_psd(series):
    """Compute Power Spectral Density (PSD)."""
    freqs, psd_vals = welch(series, fs=100, nperseg=min(len(series), window_size))
    return np.mean(psd_vals)  # Return average PSD over the window

def compute_entropy(series):
    """Compute Spectral Entropy."""
    freqs, psd_vals = welch(series, fs=100, nperseg=min(len(series), window_size))
    psd_norm = psd_vals / np.sum(psd_vals)  # Normalize PSD
    return entropy(psd_norm)  # Shannon entropy





In [156]:
df_frequency_list = []

# Loop through each unique subject_id
for s in df_temporal_features['subject_id'].unique():
    subject_data = df_temporal_features[df_temporal_features['subject_id'] == s].copy()
    
    # Loop through each unique label within the subject
    for l in subject_data['label'].unique():
        # Subset the data for the current subject_id and label, and reset the index
        subset = subject_data[subject_data['label'] == l].copy()
        # Loop through each sensor column
        for col in ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']:
            subset[f'{col}_fft'] = subset[col].rolling(window=10).apply(compute_fft, raw=True)
            subset[f'{col}_psd'] = subset[col].rolling(window=10).apply(compute_psd, raw=True)
            subset[f'{col}_entropy'] = subset[col].rolling(window=10).apply(compute_entropy, raw=True)
        
        # Add subject_id and label to the subset explicitly
        subset['subject_id'] = s
        subset['label'] = l
        
        # Append the subset to the list
        df_frequency_list.append(subset)

# Concatenate all subsets into one DataFrame
df_frequency_features = pd.concat(df_frequency_list)


In [157]:
df_frequency_features.columns

Index(['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'label',
       'subject_id', 'trial', 'age', 'height', 'weight', 'gender', 'acc_mag',
       'gyro_mag', 'acc_x_mean', 'acc_x_std', 'acc_x_skew', 'acc_x_kurtosis',
       'acc_x_jerk', 'acc_x_min', 'acc_x_max', 'acc_x_minmax_diff',
       'acc_y_mean', 'acc_y_std', 'acc_y_skew', 'acc_y_kurtosis', 'acc_y_jerk',
       'acc_y_min', 'acc_y_max', 'acc_y_minmax_diff', 'acc_z_mean',
       'acc_z_std', 'acc_z_skew', 'acc_z_kurtosis', 'acc_z_jerk', 'acc_z_min',
       'acc_z_max', 'acc_z_minmax_diff', 'gyro_x_mean', 'gyro_x_std',
       'gyro_x_skew', 'gyro_x_kurtosis', 'gyro_x_jerk', 'gyro_x_min',
       'gyro_x_max', 'gyro_x_minmax_diff', 'gyro_y_mean', 'gyro_y_std',
       'gyro_y_skew', 'gyro_y_kurtosis', 'gyro_y_jerk', 'gyro_y_min',
       'gyro_y_max', 'gyro_y_minmax_diff', 'gyro_z_mean', 'gyro_z_std',
       'gyro_z_skew', 'gyro_z_kurtosis', 'gyro_z_jerk', 'gyro_z_min',
       'gyro_z_max', 'gyro_z_minmax_diff', 'acc_x_ff

In [158]:
df_temporal_features.columns

Index(['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'label',
       'subject_id', 'trial', 'age', 'height', 'weight', 'gender', 'acc_mag',
       'gyro_mag', 'acc_x_mean', 'acc_x_std', 'acc_x_skew', 'acc_x_kurtosis',
       'acc_x_jerk', 'acc_x_min', 'acc_x_max', 'acc_x_minmax_diff',
       'acc_y_mean', 'acc_y_std', 'acc_y_skew', 'acc_y_kurtosis', 'acc_y_jerk',
       'acc_y_min', 'acc_y_max', 'acc_y_minmax_diff', 'acc_z_mean',
       'acc_z_std', 'acc_z_skew', 'acc_z_kurtosis', 'acc_z_jerk', 'acc_z_min',
       'acc_z_max', 'acc_z_minmax_diff', 'gyro_x_mean', 'gyro_x_std',
       'gyro_x_skew', 'gyro_x_kurtosis', 'gyro_x_jerk', 'gyro_x_min',
       'gyro_x_max', 'gyro_x_minmax_diff', 'gyro_y_mean', 'gyro_y_std',
       'gyro_y_skew', 'gyro_y_kurtosis', 'gyro_y_jerk', 'gyro_y_min',
       'gyro_y_max', 'gyro_y_minmax_diff', 'gyro_z_mean', 'gyro_z_std',
       'gyro_z_skew', 'gyro_z_kurtosis', 'gyro_z_jerk', 'gyro_z_min',
       'gyro_z_max', 'gyro_z_minmax_diff'],
      dt

In [159]:
df_temporal_features

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label,subject_id,trial,age,...,gyro_y_max,gyro_y_minmax_diff,gyro_z_mean,gyro_z_std,gyro_z_skew,gyro_z_kurtosis,gyro_z_jerk,gyro_z_min,gyro_z_max,gyro_z_minmax_diff
timestamp,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:38:38.700,0.948777,-9.636166,0.002699,0.003818,0.016875,0.006643,BSC,1.0,1.0,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-01 00:38:38.800,0.948993,-9.759188,0.087482,0.020693,0.027565,0.015669,BSC,1.0,1.0,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.009026,NaN,NaN,NaN
1970-01-01 00:38:38.900,1.071514,-9.787465,-0.093610,0.168721,0.075747,0.037797,BSC,1.0,1.0,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.022129,NaN,NaN,NaN
1970-01-01 00:38:39.000,1.135679,-9.754036,0.065878,0.339488,0.078970,0.040134,BSC,1.0,1.0,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.002337,NaN,NaN,NaN
1970-01-01 00:38:39.100,1.126174,-9.443248,0.075006,0.594494,-0.012065,0.103557,BSC,1.0,1.0,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.063423,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 04:49:55.200,2.128992,14.084110,0.907692,-1.022787,0.162796,0.609964,WAL,67.0,1.0,23.0,...,1.065380,2.239997,-0.128272,1.100570,1.237367,0.620927,0.912404,-1.249204,2.374952,3.624157
1970-01-01 04:49:55.300,-1.563028,11.139874,1.061856,-2.417346,-0.699792,1.861551,WAL,67.0,1.0,23.0,...,1.065380,2.239997,-0.179612,0.975716,0.957954,-0.157557,1.251587,-1.249204,1.861551,3.110755
1970-01-01 04:49:55.400,2.767679,8.869539,0.154884,-0.998184,0.612194,1.557859,WAL,67.0,1.0,23.0,...,0.612194,1.786811,-0.104783,1.082822,0.876532,-0.680915,-0.303692,-1.249204,1.861551,3.110755


### Deal with window overlap (50% Overlap)

In [163]:
# Now, subsample every `stride`-th row
df_ws10_all = df_frequency_features.iloc[::stride].dropna()

In [164]:
df_ws10_all

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label,subject_id,trial,age,...,acc_z_entropy,gyro_x_fft,gyro_x_psd,gyro_x_entropy,gyro_y_fft,gyro_y_psd,gyro_y_entropy,gyro_z_fft,gyro_z_psd,gyro_z_entropy
timestamp,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:38:39.700,-0.068994,-6.987095,5.288118,1.340131,0.254273,0.553948,BSC,1.0,1.0,32.0,...,0.552713,2.237175,0.000722,0.920616,0.363307,0.000888,1.282563,0.614725,0.000267,1.354238
1970-01-01 00:38:40.200,-0.470804,-1.801783,8.929937,1.701367,0.193919,0.328752,BSC,1.0,1.0,32.0,...,0.697146,1.886452,0.000581,0.905243,0.727568,0.000668,1.175494,1.138423,0.000603,1.415926
1970-01-01 00:38:40.700,-9.901018,13.441229,7.822155,-0.351828,0.343581,-0.851439,BSC,1.0,1.0,32.0,...,1.074713,3.329770,0.005098,1.203505,1.706857,0.000565,0.936669,3.093796,0.002285,0.577150
1970-01-01 00:38:41.200,-5.150726,6.613287,3.195224,-0.107008,-1.118601,-0.638797,BSC,1.0,1.0,32.0,...,1.461785,9.083104,0.043227,1.271543,2.602335,0.005232,1.358140,3.267480,0.006812,0.745639
1970-01-01 00:38:41.700,-4.931030,5.574285,6.261700,-0.019441,0.012431,0.115133,BSC,1.0,1.0,32.0,...,1.176317,1.104385,0.000710,1.366146,3.538939,0.009700,1.667364,0.830726,0.003255,1.440714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 04:49:53.200,0.586223,5.443400,1.029336,-0.737727,1.443459,1.335214,WAL,67.0,1.0,23.0,...,1.343248,7.967362,0.019501,0.917517,3.978832,0.003662,1.133685,9.439387,0.030438,0.913799
1970-01-01 04:49:53.700,2.286937,11.130139,-1.195925,1.260658,-0.457721,-0.981431,WAL,67.0,1.0,23.0,...,1.502760,7.253110,0.017147,0.899968,6.142548,0.022069,1.258938,8.046604,0.023517,0.994088
1970-01-01 04:49:54.200,-5.076769,14.152212,2.039380,-2.541764,-1.378478,2.412521,WAL,67.0,1.0,23.0,...,1.377740,5.479225,0.007237,1.008082,3.459003,0.006143,1.726653,5.018624,0.007999,1.087638


In [165]:
df_ws10_all.to_pickle("../../data/interim/df_81_features.pkl")